In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[ 335114.28657936,  341803.76737447],
        [ 348656.74325798,  344115.77367828],
        [ 637090.97203516,  636818.41668356],
        [ 671131.40678613,  682086.92331011],
        [ 166807.10726907,  161340.28445513],
        [ 208397.61702566,  212269.66463485],
        [ 398222.06456861,  408752.41543812],
        [ 401373.32750189,  398944.31943346],
        [ 370974.05590849,  388334.49208636],
        [ 397215.67348913,  382283.18939738],
        [ 369473.63514832,  367222.15318984],
        [ 337169.62973333,  333501.63463774],
        [ 100266.10205147,   97321.81185715],
        [ 101420.15524158,  101477.8589123 ],
        [ 162234.50875298,  151077.28824074],
        [ 146091.57782324,  141074.9812665 ],
        [ 214020.16148915,  210833.90655819],
        [ 220813.47078402,  215929.43016519],
        [ 369410.5564851 ,  367547.26200248],
        [ 355145.37875832,  346053.27198612],
        [ 526619.26909059,  523644.1817912 ],
        [ 531049.10958179,  532305

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-02, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.30025370e+07  0.00000000e+00  5.32e+03 2.20e+03  1.00e+06     0s
   1  -7.64238224e+06 -5.73938835e+06  1.03e+03 4.26e+02  2.46e+05     0s
   2  -6.13898732e+06 -8.47539833e+06  1.03e-03 4.26e-04  4.17e+04     0s
   3  -6.83367453e+06 -7.30535610e+06  7.09e-05 2.93e-05  8.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[12339.223444138632,
 152309.32971870698,
 74484.48659745885,
 6.157705451705097e-07,
 46976.8869900123,
 7.198056564927668e-05,
 6.059090315147604e-07,
 7.459762255230206e-07,
 1.0125991529094963e-09,
 1.737929939928838e-08,
 1.010935951017343e-09,
 1.0126096342838093e-09,
 1.0125858781878123e-09,
 1.017333966659257e-09,
 4.780778603511327e-06,
 2.258014072065559e-08,
 4.925123195806608e-07,
 6.158313249945003e-07,
 85874.04324624025,
 119798.11120957871,
 6.029446157746984e-07,
 44567.54317595367,
 110326.7618695317,
 6.049213875781285e-07,
 5.994649909056961e-07,
 6.093559341383307e-07,
 39435.9896500579,
 125286.80462193431,
 367356.43930539157,
 2.268857388306463e-08,
 69066.63508619973,
 27950.87239385975,
 30124.997387249336,
 98965.81031556544,
 28497.53993699823,
 9.704886269880537e-06,
 2.2634287089279786e-08,
 274080.417551923,
 1.0629906182403268e-05,
 46401.10143378265,
 521199.30414508824,
 6.061407014406792e-07,
 3.357675314834708e-06,
 2.2535205073181012e-08,
 1.6030605